In [1]:
import pandas as pd
from src.eventlog_utils import convert_log_from_lc_to_se
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from log_distance_measures.config import EventLogIDs, AbsoluteTimestampType, discretize_to_hour
from log_distance_measures.control_flow_log_distance import control_flow_log_distance
from log_distance_measures.n_gram_distribution import n_gram_distribution_distance
from log_distance_measures.absolute_event_distribution import absolute_event_distribution_distance
from log_distance_measures.case_arrival_distribution import case_arrival_distribution_distance
from log_distance_measures.circadian_event_distribution import circadian_event_distribution_distance
from log_distance_measures.relative_event_distribution import relative_event_distribution_distance
from log_distance_measures.work_in_progress import work_in_progress_distance
from log_distance_measures.cycle_time_distribution import cycle_time_distribution_distance
from log_distance_measures import earth_movers_distance
from src.res_based_ced import resource_based_circadian_event_distribution_distance
from distance_utils import emd_attributes

In [2]:
case_studies = {
    1: 'Purchasing',
    2: 'Production',
    3: 'Consulta',
    4: 'bpi12',
    5: 'bpi17',
    6: 'sepsis',
    8: 'bpi19',
    7: 'rtf'
}

In [3]:
# chose case study
case_study = case_studies[5]

# Choose if our approach or Sota
our_approach = False
Sota = not(our_approach)
approach = 'SIMOD' if Sota else None
print(f'Case study: {case_study}, our approach = {our_approach}')

Case study: bpi17, our approach = False


In [4]:
log_real = xes_importer.apply(f'data/{case_study}/logTest.xes')
log_real = pm4py.convert_to_dataframe(log_real)

if our_approach:
    log_sim = pd.read_csv(f'simulations/{case_study}/sim_0.csv')
    print('imported log from our approach')

elif Sota:
    
    if approach == 'RIMS':
        log_sim = pd.read_csv(f'RIMS/{case_study}/results/rims/sim.csv')
        del log_sim['st_wip']
        del log_sim['queue']
        del log_sim['st_tsk_wip']
        print('imported log from Sota, approach:', approach)        
    
    elif approach == 'AgentSimulator':
        log_sim = pd.read_csv(f'AgentSimulator/{case_study}/main_results/sim.csv')
        print('imported log from Sota, approach:', approach)    

    elif approach == 'SIMOD':
        log_sim = pd.read_csv(f'SIMOD_TEST/{case_study}/output/best_result/evaluation/simulated_log_0.csv')
        print('imported log from Sota, approach:', approach)

    elif approach == 'DSIM':
        log_sim = pd.read_csv(f'DSIM/results/generated_logs/{case_study}/DSIM/sim.csv')
        print('imported log from Sota, approach:', approach)

    else:
        log_sim = pd.read_csv(f'Sota/{case_study}/{approach}/sim.csv')
    
    #Rename the columns named 'end_timestamp' and 'start_timestamp' with 'time:timestamp' and 'start:timestamp'
    log_sim = log_sim.rename(columns={'end_timestamp': 'time:timestamp', 
                                      'start_timestamp': 'start:timestamp', 
                                      'task':'concept:name',
                                      'caseid': 'case:concept:name',
                                      'resource': 'org:resource'}, errors='ignore')
    
    # Errors='ignore' has been set because the log from francesca meneghello et al has some columns already correctly

    print('imported log from Sota, approach:', approach, ',columns renamed')

parsing log, completed traces ::   0%|          | 0/5771 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'SIMOD_TEST/bpi17/output/best_result/evaluation/simulated_log_0.csv'

In [ ]:
# convert event log format lifecycles
if 'lifecycle:transition' in log_real.columns:
    log_real = convert_log_from_lc_to_se(log_real)
    log_real.rename(columns={'START': 'start:timestamp', 'END': 'time:timestamp'}, errors='ignore', inplace=True)
    log_real.reset_index(inplace=True)
if 'lifecycle:transition' in log_sim.columns:
    log_sim = convert_log_from_lc_to_se(log_sim)
    log_sim.reset_index(inplace=True)

NameError: name 'log_real' is not defined

In [ ]:
# Set event log column ID mapping
event_log_ids = EventLogIDs(
    case="case:concept:name",
    activity="concept:name",
    start_time="start:timestamp",
    end_time="time:timestamp",
    resource="org:resource",
)

In [ ]:
log_real[event_log_ids.start_time] = pd.to_datetime(log_real[event_log_ids.start_time], utc=True)
log_real[event_log_ids.end_time] = pd.to_datetime(log_real[event_log_ids.end_time], utc=True)

# log_sim[event_log_ids.start_time] = [i[:19] for i in log_sim[event_log_ids.start_time].values]
# log_sim[event_log_ids.end_time] = [i[:19] for i in log_sim[event_log_ids.end_time].values]
# log_sim[event_log_ids.start_time] = pd.to_datetime(log_sim[event_log_ids.start_time], utc=True)
# log_sim[event_log_ids.end_time] = pd.to_datetime(log_sim[event_log_ids.end_time], utc=True)

In [ ]:
#Initialize distances dictionary
distances = {}

In [ ]:
from log_distance_measures.circadian_workforce_distribution import circadian_workforce_distribution_distance
print('case study is: ', case_study, 'CWD is: ', circadian_workforce_distribution_distance(log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
))

### Generateed Attributes Distribution Distance

In [ ]:
distance = emd_attributes(log_real, log_sim, attr_names=[])
print('Attr distance: ', distance)
# Fill the distances dictionary
distances['emd_attributes'] = distance

Attr distance:  nan


<b>WARNING: It may take a long time</b>

### Control-flow Log Distance

In [ ]:
# Call passing the event logs, and its column ID mappings
distance = control_flow_log_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
)

In [ ]:
print('CF Log distance: ', distance)

# Fill the distances dictionary
distances['control_flow_log_distance'] = distance


### N-Gram Distribution Distance

In [ ]:
n_gram = 3

In [ ]:
# Call passing the event logs, and its column ID mappings
distance = n_gram_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    n=n_gram
)

In [ ]:
print('N-Gram distr. distance: ', distance)
# Fill the distances dictionary
distances['n_gram_distribution_distance'] = distance

### Absolute Event Distribution Distance

In [ ]:
# EMD of the (END) timestamps distribution where each bin represents a minute
distance = absolute_event_distribution_distance(
    log_real, event_log_ids,
    log_sim, event_log_ids,
    discretize_type=AbsoluteTimestampType.END,
    discretize_event=discretize_to_hour
)

In [ ]:
print('Absolute Event Distribution Distance: ', distance)
# Fill the distances dictionary
distances['absolute_event_distribution_distance'] = distance

### Case Arrival Distribution Distance

In [ ]:
distance = case_arrival_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_event=discretize_to_hour  # Function to discretize each timestamp (default by hour)
)

In [ ]:
print('Case Arrival distr distance: ', distance)
# Fill the distances dictionary
distances['case_arrival_distribution_distance'] = distance

### Circadian Event Distribution Distance

In [ ]:
distance = circadian_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
)

In [ ]:
print('Circadian Event distr distance: ', distance)
# Fill the distances dictionary 
distances['circadian_event_distribution_distance'] = distance

### Resource-Based Circadian Event Distribution Distance


In [ ]:
distance = resource_based_circadian_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH  # Consider both start/end timestamps of each activity instance
)

In [ ]:
print('Resource-Based Circadian Event distr distance: ', distance)
# Fill the distances dictionary 
distances['resource_based_circadian_event_distribution_distance'] = distance

### Relative Event Distribution Distance

In [ ]:
# Call passing the event logs, its column ID mappings, timestamp type, and discretize function
distance = relative_event_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    discretize_type=AbsoluteTimestampType.BOTH,  # Type of timestamp distribution (consider start times and/or end times)
    discretize_event=discretize_to_hour  # Function to discretize the absolute seconds of each timestamp (default by hour)
)

In [ ]:
print('Relative Event distr distance: ', distance)
# Fill the distances dictionary
distances['relative_event_distribution_distance'] = distance

### Work in Progress Distance

<b>WARNING: It may take a long time</b>

In [ ]:
# # Call passing the event logs, its column ID mappings, timestamp type, and discretize function
# distance = work_in_progress_distance(
#     log_real, event_log_ids,  # First event log and its column id mappings
#     log_sim, event_log_ids,  # Second event log and its column id mappings
#     window_size=pd.Timedelta(hours=1)  # Bins of 1 hour
# )

In [ ]:
# print('Work in Progress distance: ', distance)
# Fill the distances dictionary
# distances['work_in_progress_distance'] = distance

### Cycle Time Distribution Distance

In [ ]:
distance = cycle_time_distribution_distance(
    log_real, event_log_ids,  # First event log and its column id mappings
    log_sim, event_log_ids,  # Second event log and its column id mappings
    bin_size=pd.Timedelta(hours=1)  # Bins of 1 minute
)

In [ ]:
print('Cycle Time distr distance: ', distance)
# Fill the distances dictionary
distances['cycle_time_distribution_distance'] = distance

### Save them

In [ ]:
print(f'case study is {case_study} for approach {approach}, with our approach {our_approach} and Sota {Sota}')
# Save the distances dictionary to a file with pkl
pd.to_pickle(distances, f'simulations/{case_study}/metrics.pkl')

# Print the distances vector with each distance measure, in different lines
for key, value in distances.items():
    print(key, value)



### Cast them into .xes for evaluating entropy

In [5]:
import os 
directory = '/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data'
os.chdir(directory)

import glob
from src.eventlog_utils import convert_log_from_lc_to_se
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
print('Folder s in directory:', directory)
for folder in glob.glob('*'):  # Skip the first folder which is the current directory
    if os.path.isdir(folder):
        print(folder)
        for file in glob.glob(f'{folder}/*.xes'):
            log = xes_importer.apply(file)
            log = pm4py.convert_to_dataframe(log)
            if 'lifecycle:transition' in log.columns:
                log = convert_log_from_lc_to_se(log)
                log.rename(columns={'START': 'start:timestamp', 'END': 'time:timestamp'}, errors='ignore', inplace=True)
                log.reset_index(inplace=True)
            # Save the log to a csv file
            log.to_csv(f'{folder}/{os.path.basename(file).replace(".xes", ".csv")}', index=False)
            print(f'Saved {file} to {folder}/{os.path.basename(file).replace(".xes", ".csv")}')

Folder s in directory: /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data
Production


parsing log, completed traces ::   0%|          | 0/180 [00:00<?, ?it/s]

100%|██████████| 180/180 [00:03<00:00, 45.99it/s]

Saved Production/logTrain.xes to Production/logTrain.csv


parsing log, completed traces ::   0%|          | 0/45 [00:00<?, ?it/s]

...

100%|██████████| 608/608 [00:08<00:00, 73.00it/s] 


Saved Purchasing/PurchasingExample.xes to Purchasing/PurchasingExample.csv
rtf


parsing log, completed traces ::   0%|          | 0/120296 [00:00<?, ?it/s]

 34%|███▍      | 40700/120296 [25:40<50:13, 26.41it/s]  


KeyboardInterrupt: 

In [6]:
import time
import subprocess

case_studies = {
    1: 'Purchasing',
    2: 'Production',
    3: 'Consulta',
    4: 'bpi12',
    5: 'bpi17',
    6: 'sepsis',
    7: 'rtf',
    8: 'bpi19'
}

base_path = "/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simod_new"

for key, case_name in case_studies.items():
    config_path = f"{base_path}/{case_name}.yml"
    command = ["simod", "--configuration", config_path]
    
    print(f"\nRunning case study: {case_name}")
    
    start_time = time.time()
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error while running {case_name}: {e}")
    end_time = time.time()
    
    duration = end_time - start_time
    print(f"Execution time for {case_name}: {duration:.2f} seconds")


Running case study: Purchasing

Pre-processing

Adjusting timestamps for multitasking

Adding enabled times

Discovering initial BPS Model

Optimizing control-flow parameters
Output file already exists, it would be overwritten


INFO:root:Running java -jar /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/.conda/lib/python3.11/site-packages/lib/openxes-cli.jar -f /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/outputs/20250619_174709_B76D4AD0_6EFB_4B55_A8B0_F6C90FA1773C/control-flow/logTrain.xes -t xes -o /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/outputs/20250619_174709_B76D4AD0_6EFB_4B55_A8B0_F6C90FA1773C/control-flow/logTrain.xes


start serializing log to XES.XML
finished serializing log (95 msec.)


INFO:root:Running java -jar /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/.conda/lib/python3.11/site-packages/lib/openxes-cli.jar -f /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/outputs/20250619_174709_B76D4AD0_6EFB_4B55_A8B0_F6C90FA1773C/control-flow/logTrain_only_end.xes -t xes -o /home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/outputs/20250619_174709_B76D4AD0_6EFB_4B55_A8B0_F6C90FA1773C/control-flow/logTrain_only_end.xes


Output file already exists, it would be overwritten
start serializing log to XES.XML
finished serializing log (46 msec.)

Control-flow optimization iteration 0
-------------------------------------
Parameters: HyperoptIterationParams(output_dir=PosixPath('/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/data/outputs/20250619_174709_B76D4AD0_6EFB_4B55_A8B0_F6C90FA1773C/control-flow/iteration_20250619_174715_DE69E28C_2E7F_4078_9EF0_E8428EC9EAB1'), provided_model_path=None, project_name='logTrain', optimization_metric=<Metric.THREE_GRAM_DISTANCE: 'three_gram_distance'>, gateway_probabilities_method=<GatewayProbabilitiesDiscoveryMethod.DISCOVERY: 'discovery'>, mining_algorithm=<ProcessModelDiscoveryAlgorithm.SPLIT_MINER_V1: 'sm1'>, epsilon=0.24264130647471222, eta=0.2709400151247193, replace_or_joins='True', prioritize_parallelism='False', f_score=0.8434001858906655)

Discovering Process Model with sm1

SplitMiner v1 is running with the following arguments: ['java', '-Xmx2G', '-Xms

...

Error while running sepsis: Command '['simod', '--configuration', '/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simod_new/sepsis.yml']' returned non-zero exit status 1.
Execution time for sepsis: 3.81 seconds

Running case study: rtf


Usage: simod [OPTIONS]
Try 'simod --help' for help.

Error: Invalid value for '--configuration' / '-c': File '/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simod_new/rtf.yml' does not exist.


Error while running rtf: Command '['simod', '--configuration', '/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simod_new/rtf.yml']' returned non-zero exit status 2.
Execution time for rtf: 0.90 seconds

Running case study: bpi19

Pre-processing

Adding start times


concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/.conda/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3812, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas/_libs/index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7096, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'start:timestamp'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/.conda/lib/python3.11/concurrent/futures/process.py", line 261, in _process_w

Error while running bpi19: Command '['simod', '--configuration', '/home/padela/Scrivania/ProbabilityBasedEventLogGenerator/simod_new/bpi19.yml']' returned non-zero exit status 1.
Execution time for bpi19: 98.32 seconds
